# Alembic Table Primer

In this note the `alembic` database migration tool is introduced. Alembic is closely used with SQLAlchemy database toolkit for Python.

Goals:

- reproduce the previous tutorial in the context of alembic migrations;

- moving from in-memory to file-oriented SQLite databases;

**Attention**: this notebook uses the Python interpreter from the virtual environment. It is doable via selecting the kernel (programming language specific processes). Read more about kernles at the [Jupyter docs](https://docs.jupyter.org/en/latest/projects/kernels.html).

## Table of Contents

- [Installing Alembic](#installing-alembic)

- [The Migration Environment](#the-migration-environment)

- [The First Migration Script](#the-first-migration-script)

- [The Second Migration Script](#the-second-migration-script)

In [1]:
import sqlalchemy as sqla
import sqlalchemy.ext.asyncio as asqla


print(f"SQLAlchemy version: {sqla.__version__}")

SQLAlchemy version: 2.0.22


In [2]:
url = "sqlite+aiosqlite:///db.sqlite"

async_engine = asqla.create_async_engine(url)

### Installing Alembic

Alembic installation (read more on [front matter page](https://alembic.sqlalchemy.org/en/latest/front.html)) is as simple as installing practically any Python package - just `pip`ping it down.

```shell
(sqla) ╭─<...@...> ~/Projects/SQL-via-Alchemy  ‹02_alembic*›
╰─➤  pip3 install alembic
Collecting alembic
  Downloading alembic-1.12.1-py3-none-any.whl.metadata (7.3 kB)
Requirement already satisfied: SQLAlchemy>=1.3.0 in ./sqla/lib/python3.10/site-packages (from alembic) (2.0.22)
Collecting Mako (from alembic)
  Downloading Mako-1.3.0-py3-none-any.whl.metadata (2.9 kB)
Requirement already satisfied: typing-extensions>=4 in ./sqla/lib/python3.10/site-packages (from alembic) (4.8.0)
Requirement already satisfied: greenlet!=0.4.17 in ./sqla/lib/python3.10/site-packages (from SQLAlchemy>=1.3.0->alembic) (3.0.0)
Requirement already satisfied: MarkupSafe>=0.9.2 in ./sqla/lib/python3.10/site-packages (from Mako->alembic) (2.1.3)
Downloading alembic-1.12.1-py3-none-any.whl (226 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 2.1 MB/s eta 0:00:00
Downloading Mako-1.3.0-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00
Installing collected packages: Mako, alembic
Successfully installed Mako-1.3.0 alembic-1.12.1
```

### The Migration Environment

The migration environment is a directory where the database migration scripts are stored. Database migrations, also known as schema migrations, or just migrations are sets of changes to modify the structure of the entities within a relational database (RDB). This means that a migration is a transition of an RDB from one state to another.

Why care about migrations here and now? Because we can reproduce the previous note steps:

- create a database and even populate it with data, but the latter seems to be discourageable - the first migration;

- change the structure of the table created in the first migration and here we need to process saved data and pour them into the new table - the second migration;

This is practical and that is the reason to care. This [article](https://www.prisma.io/dataguide/types/relational/what-are-database-migrations) helps in understanding migrations.

The first step is to initialise the migration environment.

In [3]:
%%bash
alembic init --help

usage: alembic init [-h] [-t TEMPLATE] [--package] directory

positional arguments:
  directory             location of scripts directory

options:
  -h, --help            show this help message and exit
  -t TEMPLATE, --template TEMPLATE
                        Setup template for use with 'init'
  --package             Write empty __init__.py files to the environment and
                        version locations


The major part is choosing a template for the project migration environment structure.

In [4]:
%%bash
alembic list_templates

Available templates:

generic - Generic single-database configuration.
multidb - Rudimentary multi-database configuration.
async - Generic single-database configuration with an async dbapi.

Templates are used via the 'init' command, e.g.:

  alembic init --template generic ./scripts


Nice, and it is so kind to give a useful piece of advice.

In [5]:
%env MIG_DIR=./migenv
%env DB_FILE=./db.sqlite

env: MIG_DIR=./migenv
env: DB_FILE=./db.sqlite


In [6]:
%%bash
echo "Migration directory: " ${MIG_DIR}
rm -rf ${MIG_DIR}
rm -rf ${DB_FILE}

alembic init --template async ${MIG_DIR}

Migration directory:  ./migenv
Creating directory '/home/stankudrow/Projects/SQL-via-Alchemy/tutorial/02_table_primer_alembic/migenv' ...  done
Creating directory '/home/stankudrow/Projects/SQL-via-Alchemy/tutorial/02_table_primer_alembic/migenv/versions' ...  done
File '/home/stankudrow/Projects/SQL-via-Alchemy/tutorial/02_table_primer_alembic/alembic.ini' already exists, skipping
Generating /home/stankudrow/Projects/SQL-via-Alchemy/tutorial/02_table_primer_alembic/migenv/script.py.mako ...  done
Generating /home/stankudrow/Projects/SQL-via-Alchemy/tutorial/02_table_primer_alembic/migenv/README ...  done
Generating /home/stankudrow/Projects/SQL-via-Alchemy/tutorial/02_table_primer_alembic/migenv/env.py ...  done
Please edit configuration/connection/logging settings in '/home/stankudrow/Projects/SQL-via-Alchemy/tutorial/02_table_primer_alembic/alembic.ini' before proceeding.


Listing the contents of this note directory must show the following files (directories are also files, at least in Linux, no idea about Windows):

- migration environment configuration file - `alembic.ini`

- migration environment directory

In [7]:
%%bash
ls

02_table_primer_alembic.ipynb
alembic.ini
migenv


In [8]:
%%bash
ls ${MIG_DIR}

env.py
README
script.py.mako
versions


Extraction from the [alembic tutorial](https://alembic.sqlalchemy.org/en/latest/tutorial.html):

- env.py - a Python script that is run whenever the alembic migration tool is invoked.

- README - included with the various environment templates, should have something informative.

- script.py.mako - a Mako template file which is used to generate new migration scripts. Whatever is here is used to generate new files within versions/ directory.

- versions/ - the directory holds the individual version scripts.

### The First Migration Script

Alembic helps to create a migration script template which is called a revision.

Let's create a "Users" table within the migration script so the table has the following structure:

```sql
CREATE TABLE users (
	"id" INTEGER NOT NULL, 
	"full name" VARCHAR(100) NOT NULL, 
	PRIMARY KEY ("id")
)
```

In [9]:
%%bash
alembic revision -m "create users table"
ls ${MIG_DIR}/versions

Generating /home/stankudrow/Projects/SQL-via-Alchemy/tutorial/02_table_primer_alembic/migenv/versions/27fc5c72440c_create_users_table.py ...  done
27fc5c72440c_create_users_table.py
__pycache__


Try to rerun the previous cell code: no error will be emitted, just a new migration script will be created.

The `file_template` name is composed according to the following scheme - %%(rev)s_%%(slug)s - where:

- %%(rev)s - revision id

- %%(slug)s - a truncated string derived from the revision message

- other parts, mostly about date-time, feel free to read more in the [documentaion](https://alembic.sqlalchemy.org/en/latest/tutorial.html).

The first migration script turn to be as follows:

```python
"""create users table

Revision ID: alembic-generated-revision-id
Revises: 
Create Date: 2023-11-19 20:13:03.526758

"""
from typing import Sequence, Union

from alembic import op
import sqlalchemy as sa


# revision identifiers, used by Alembic.
revision: str = 'alembic-generated-revision-id'
down_revision: Union[str, None] = None
branch_labels: Union[str, Sequence[str], None] = None
depends_on: Union[str, Sequence[str], None] = None


USERS: str = "users"


def upgrade() -> None:
    op.create_table(
        USERS,
        sa.Column("id", sa.Integer, primary_key=True),
        sa.Column("full name", sa.Unicode(100)),
    )


def downgrade() -> None:
    op.drop_table(
        USERS,
    )
```

Time to upgrade the database by running the migration script. But before doing, a slight change in alembic.ini file is vital:

```
sqlalchemy.url = sqlite+aiosqlite:///db.sqlite
```

**Note**: if you have rerun this notebook, consider to add `upgrade` and `downgrade` logic in the generated by alembic migration script. And only after the changes you can proceed without heckups. Otherwise you'll have a migration with non-defined upgrade/downgrade logic, so the further steps will be useless.

In [10]:
%%bash
alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 27fc5c72440c, create users table


Time to play with the newly migrated database.

In [11]:
metadata = sqla.MetaData()


def reflect_table(conn_, name: str) -> sqla.Table:
    """Reflects the table `name` from the DB via connection `conn_`."""

    return sqla.Table(
        name,
        sqla.MetaData(),
        autoload_with=conn_
    )


async with async_engine.connect() as conn:
    users_table: sqla.Table = await conn.run_sync(
        reflect_table, "users"
    )
    print(f"Table name -> {users_table.name}")
    print(f"Table columns -> {users_table.c.items()}")
    print(f"Selecting data from {users_table} table.")

Table name -> users
Table columns -> [('id', Column('id', INTEGER(), table=<users>, primary_key=True, nullable=False)), ('full name', Column('full name', VARCHAR(length=100), table=<users>))]
Selecting data from users table.


Yeah, the "users" table has been reflected.

Now the plan is as plain as a plane: insert some data.

In [12]:
async with async_engine.begin() as conn:
    result = await conn.execute(
        sqla.insert(users_table),
        [
            {
                "full name": "John Doe"
            },
            {
                "full name": "Jane Doe"
            }
        ]
    )

The previous cell can be rerun: passing only the same "full name" attribute values will not cause violations since the primary key attribute gets autoincremented and the rows get unique.

Consider executing the previous INSERT cell to get quasi-duplicates in the next SELECT code. That will be give a weak reason for downgrading migration. Certainly, the actual reason for making downgrade is just to illustrate this option, just in case.

In [13]:
async with async_engine.connect() as conn:
    result = await conn.execute(
        sqla.select(users_table)
    )
    for nth, row in enumerate(result, 1):
        print(f"Row {nth}: {row}")

Row 1: (1, 'John Doe')
Row 2: (2, 'Jane Doe')


The history of the migrations^ should be from \<base\> (meaning "from scratch") to the first revision id.

In [14]:
%%bash
alembic history

<base> -> 27fc5c72440c (head), create users table


Let's downgrade the migration: for now it means to get back to the clear state since this migration is the first one.

In [15]:
%%bash
alembic downgrade --help

usage: alembic downgrade [-h] [--sql] [--tag TAG] revision

positional arguments:
  revision    revision identifier

options:
  -h, --help  show this help message and exit
  --sql       Don't emit SQL to database - dump to standard output/file
              instead. See docs on offline mode.
  --tag TAG   Arbitrary 'tag' name - can be used by custom env.py scripts.


But you can just put the revision id of the first migration script - it is already in effect. We need to downgrade to the initial (base) state, but it has down_revision (see the migration script) set to None. The "Stack Overflow" comes to rescue with ["undoing the last alembic migration"](https://stackoverflow.com/questions/48242324/undo-last-alembic-migration) question, but it is also covered in the [alembic tutorial](https://alembic.sqlalchemy.org/en/latest/tutorial.html).

In [16]:
%%bash
alembic downgrade base

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running downgrade 27fc5c72440c -> , create users table


In [17]:
async with async_engine.connect() as conn:
    result = await conn.execute(
        sqla.select(users_table)
    )
    for nth, row in enumerate(result, 1):
        print(f"Row {nth}: {row}")

OperationalError: (sqlite3.OperationalError) no such table: users
[SQL: SELECT users.id, users."full name" 
FROM users]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Yeah, that worked, finally.

### The Second Migration Script

The second migration is meant to change the first migration changes. Since the previous chapter ends with a complete downgrade to the nothing state, we need to restore the first migration, populate the table with the data and then get the hands dirty - there will be enough work to do.

In [18]:
%%bash
alembic history

<base> -> 27fc5c72440c (head), create users table


In [19]:
%%bash
alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 27fc5c72440c, create users table


In [20]:
async with async_engine.begin() as conn:
    result = await conn.execute(
        sqla.insert(users_table),
        [
            {
                "full name": "John Doe"
            },
            {
                "full name": "Jane Doe"
            }
        ]
    )
    print(f"SELECT")
    result = await conn.execute(
        sqla.select(users_table)
    )
    for nth, row in enumerate(result, 1):
        print(f"Row {nth}: {row}")

SELECT
Row 1: (1, 'John Doe')
Row 2: (2, 'Jane Doe')


The good thing is that we can address user_table: so many changes made, but the mapping has not been corrupted (yet).

Now we coming to the interesting part, let's have a quick memo:

- the second migration should split the "full name" column into two attributes: "fname" (first name) and "lname" (last name) respectively;

- no data must be lost, we should think about them as data in the production environment

The [Alembic Cookbook](https://alembic.sqlalchemy.org/en/latest/cookbook.html) has a good example, but here we shall achieve the desired result manually.

In [21]:
%%bash
alembic current

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


27fc5c72440c (head)


In [22]:
await async_engine.dispose()

### References

- [Alembic documentation: main page](https://alembic.sqlalchemy.org/en/latest/index.html)

- [Alembic: front matter = quickstart](https://alembic.sqlalchemy.org/en/latest/front.html)

- [Alembic: tutorial](https://alembic.sqlalchemy.org/en/latest/tutorial.html)

- [The insert() SQL Expression Construct](https://docs.sqlalchemy.org/en/20/tutorial/data_insert.html)
